## Evaluate Approaches

This notebook is intended to evaluate the subset approaches for esnli, where the
output is a dataframe that looks something like:

| Classifier | Approach              | Coverage (% identified) | Validity (% identified and leads to flip) | Median Size |
|------------|-----------------------|--------------------------|--------------------------------------------|-------------|
| Random     | Class Exclusion       | x                        | x                                          | x           |
| Logistic   | Fast                  |                          |                                            |             |
| Logistic   | Slow                  |                          |                                            |             |
| Logistic   | Fast + CE fallback    |                          |                                            |             |
| Logistic   | Slow + CE fallback    |                          |                                            |             |
| KNN        | Greedy                |                          |                                            |             |
| KNN        | Greedy + CE fallback  |                          |                                            |             |
| SVM        | Greedy                |                          |                                            |             |
| SVM        | Greedy + CE fallback  |                          |                                            |             |
| DT         | Greedy                |                          |                                            |             |
| DT         | Greedy + CE fallback  |                          |                                            |             |
| LMeans     | Greedy                |                          |                                            |             |
| LMeans     | Greedy + CE fallback  |                          |                                            |             |


In [22]:
%load_ext autoreload
%autoreload 2


from typing import List
import numpy as np


DATASET_NAME = "esnli"
LABEL_SPACE = ["entailment", "neutral", "contradiction"]
MODEL_NAME = "deberta_large"
SEED = 42
POOLER = "mean_with_attention"
LAYER = 24

# check of the non_empty sets, how many are actually valid 
def summarize_subset_results(flip_list: List[List[int]], is_valid: List[bool]):
    total = len(flip_list)
    valid = 0
    identified_subset = 0
    subset_sizes = []
    for i, l in enumerate(flip_list):
        if l is not None and len(l) > 0: 
            identified_subset += 1
            if is_valid[i]:
                valid += 1
                subset_sizes.append(len(l))
    metrics = {
        "Coverage": round(identified_subset / total * 100, 2),
        "Validity": round(valid / total * 100, 2),
        "Median Valid Subset Sizes": np.median(np.array(subset_sizes))
    }
    print(f"Identified {identified_subset}/{total} subsets")
    print(f"{valid}/{identified_subset} identified subsets are valid")
    print(f"Overall, validity is {valid}/{total}, or {metrics['Validity']}%")
    print(f"Median Valid Subset Sizes is {metrics['Median Valid Subset Sizes']}")

    return metrics

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
## Load Embeddings
from utils.io import (
    load_dataset_from_hf,
    load_labels_at_split,
    load_embeddings,
    load_wrapperbox
)
import numpy as np


train_embeddings = load_embeddings(
    dataset=DATASET_NAME,
    model=MODEL_NAME,
    seed=SEED,
    split="train",
    pooler=POOLER,
    layer=LAYER
)

eval_embeddings = load_embeddings(
    dataset=DATASET_NAME,
    model=MODEL_NAME,
    seed=SEED,
    split="eval",
    pooler=POOLER,
    layer=LAYER
)

test_embeddings = load_embeddings(
    dataset=DATASET_NAME,
    model=MODEL_NAME,
    seed=SEED,
    split="test",
    pooler=POOLER,
    layer=LAYER
)

train_eval_embeddings = np.vstack([train_embeddings, eval_embeddings])


## Load Datasets and Labels
dataset = load_dataset_from_hf(dataset=DATASET_NAME)
train_labels = load_labels_at_split(dataset, "train")
eval_labels = load_labels_at_split(dataset, "eval")
train_eval_labels = np.concatenate([train_labels, eval_labels])
test_labels = load_labels_at_split(dataset, "test")

from datasets import DatasetDict, concatenate_datasets
train_eval_dataset = concatenate_datasets([dataset["train"], dataset["eval"]])
dataset_dict = DatasetDict(
    {"train": train_eval_dataset, "test": dataset["test"]}
)

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/samsoup/.cache/huggingface/token
Login successful


/home/samsoup/anaconda3/envs/wrapperbox/lib/python3.9/site-packages/datasets/load.py:2516: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(


In [6]:
## Do metrics for Yang fast

from MinimalSubsetToFlipPredictions.evaluate import evaluate_predictions
import pickle

wrapper_name = "LogisticRegression"
clf = load_wrapperbox(
    dataset=DATASET_NAME,
    model=MODEL_NAME,
    seed=SEED,
    pooler=POOLER,
    wrapperbox=wrapper_name
)

filename = "esnli_deberta_large_yang2023_fast.pickle"
with open(filename, 'rb') as handle:
    flip_list = pickle.load(handle)

ex_indices_to_check = np.arange(len(flip_list))
is_valid = evaluate_predictions(
    clf=clf,
    flip_list=flip_list,
    train_embeddings=train_eval_embeddings,
    train_labels=train_eval_labels,
    test_embeddings=test_embeddings,
    ex_indices_to_check=ex_indices_to_check,
)

metrics = summarize_subset_results(
    flip_list=flip_list,
    is_valid=is_valid
)

metrics

/home/samsoup/anaconda3/envs/wrapperbox/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
0it [00:00, ?it/s]

Of the 0 proposed subsets, only 0.0 is valid
Validity: 0.00%


In [26]:
## Do metrics for Yang slow

from MinimalSubsetToFlipPredictions.evaluate import evaluate_predictions
import pickle

wrapper_name = "LogisticRegression"
clf = load_wrapperbox(
    dataset=DATASET_NAME,
    model=MODEL_NAME,
    seed=SEED,
    pooler=POOLER,
    wrapperbox=wrapper_name
)

filename = "esnli_deberta_large_yang2023_slow.pickle"
with open(filename, 'rb') as handle:
    flip_list = pickle.load(handle)

ex_indices_to_check = np.arange(len(flip_list))
is_valid = evaluate_predictions(
    clf=clf,
    flip_list=flip_list,
    train_embeddings=train_eval_embeddings,
    train_labels=train_eval_labels,
    test_embeddings=test_embeddings,
    ex_indices_to_check=ex_indices_to_check,
)

metrics = summarize_subset_results(
    flip_list=flip_list,
    is_valid=is_valid
)

metrics

In [ ]:
## Do metrics for KNN

import json
import pprint
from MinimalSubsetToFlipPredictions.evaluate import evaluate_predictions
import pickle

wrapper_name = "KNN"
clf = load_wrapperbox(
    dataset=DATASET_NAME,
    model=MODEL_NAME,
    seed=SEED,
    pooler=POOLER,
    wrapperbox=wrapper_name
)

filename = f"esnli_deberta_large_{wrapper_name}.pickle"
with open(filename, 'rb') as handle:
    flip_list = pickle.load(handle)

total = len(flip_list)
ex_indices_to_check = np.arange(total)
flip_list = flip_list[:total]

is_valid = evaluate_predictions(
    clf=clf,
    flip_list=flip_list,
    train_embeddings=train_eval_embeddings,
    train_labels=train_eval_labels,
    test_embeddings=test_embeddings,
    ex_indices_to_check=ex_indices_to_check,
)

metrics = summarize_subset_results(
    flip_list=flip_list,
    is_valid=is_valid
)

pprint(metrics)

# Path to the output JSON file
output_file_path = f'{DATASET_NAME}_{MODEL_NAME}_{wrapper_name}_greedy_metrics.json'

# Save the dictionary to a JSON file
with open(output_file_path, 'w') as json_file:
    json.dump(metrics, json_file, indent=4)

100%|██████████| 10/10 [00:14<00:00,  1.46s/it]


[True, True, True, True, True, True, True, True, True, True]

In [24]:
## Do metrics for LGBM

from MinimalSubsetToFlipPredictions.evaluate import evaluate_predictions
import pickle

wrapper_name = "LGBM"
clf = load_wrapperbox(
    dataset=DATASET_NAME,
    model=MODEL_NAME,
    seed=SEED,
    pooler=POOLER,
    wrapperbox=wrapper_name
)

filename = f"esnli_deberta_large_{wrapper_name}.pickle"
with open(filename, 'rb') as handle:
    flip_list = pickle.load(handle)

total = len(flip_list)
ex_indices_to_check = np.arange(total)
flip_list = flip_list[:total]

is_valid = evaluate_predictions(
    clf=clf,
    flip_list=flip_list,
    train_embeddings=train_eval_embeddings,
    train_labels=train_eval_labels,
    test_embeddings=test_embeddings,
    ex_indices_to_check=ex_indices_to_check,
)

metrics = summarize_subset_results(
    flip_list=flip_list,
    is_valid=is_valid
)

pprint(metrics)

# Path to the output JSON file
output_file_path = f'{DATASET_NAME}_{MODEL_NAME}_{wrapper_name}_greedy_metrics.json'

# Save the dictionary to a JSON file
with open(output_file_path, 'w') as json_file:
    json.dump(metrics, json_file, indent=4)

  0%|          | 0/9824 [00:00<?, ?it/s]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 2.470616 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 261120
[LightGBM] [Info] Number of data points in the train set: 558761, number of used features: 1024
[LightGBM] [Info] Start training from score -1.097114
[LightGBM] [Info] Start training from score -1.101229
[LightGBM] [Info] Start training from score -1.097500


  0%|          | 21/9824 [00:16<2:05:12,  1.30it/s]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 2.408087 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 261120
[LightGBM] [Info] Number of data points in the train set: 558978, number of used features: 1024
[LightGBM] [Info] Start training from score -1.096966
[LightGBM] [Info] Start training from score -1.100971
[LightGBM] [Info] Start training from score -1.097904


  1%|          | 67/9824 [00:31<1:11:51,  2.26it/s]

...

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 2.679754 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 261120
[LightGBM] [Info] Number of data points in the train set: 558931, number of used features: 1024
[LightGBM] [Info] Start training from score -1.097053
[LightGBM] [Info] Start training from score -1.100693
[LightGBM] [Info] Start training from score -1.098094


  4%|▍         | 381/9824 [02:42<1:22:29,  1.91it/s]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 2.424240 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 261120
[LightGBM] [Info] Number of data points in the train set: 558860, number of used features: 1024
[LightGBM] [Info] Start training from score -1.096192
[LightGBM] [Info] Start training from score -1.100959
[LightGBM] [Info] Start training from score -1.098691


  4%|▍         | 391/9824 [03:10<1:16:33,  2.05it/s]


KeyboardInterrupt: 

In [ ]:
## Do metrics for LMeans

from MinimalSubsetToFlipPredictions.evaluate import evaluate_predictions
import pickle

wrapper_name = "LMeans"
clf = load_wrapperbox(
    dataset=DATASET_NAME,
    model=MODEL_NAME,
    seed=SEED,
    pooler=POOLER,
    wrapperbox=wrapper_name
)

filename = f"esnli_deberta_large_{wrapper_name}.pickle"
with open(filename, 'rb') as handle:
    flip_list = pickle.load(handle)

total = len(flip_list)
ex_indices_to_check = np.arange(total)
flip_list = flip_list[:total]

is_valid = evaluate_predictions(
    clf=clf,
    flip_list=flip_list,
    train_embeddings=train_eval_embeddings,
    train_labels=train_eval_labels,
    test_embeddings=test_embeddings,
    ex_indices_to_check=ex_indices_to_check,
)

metrics = summarize_subset_results(
    flip_list=flip_list,
    is_valid=is_valid
)

pprint(metrics)

# Path to the output JSON file
output_file_path = f'{DATASET_NAME}_{MODEL_NAME}_{wrapper_name}_greedy_metrics.json'

# Save the dictionary to a JSON file
with open(output_file_path, 'w') as json_file:
    json.dump(metrics, json_file, indent=4)

FileNotFoundError: [Errno 2] No such file or directory: 'esnli_deberta_large_LMeans.part_1.pickle'